출처<br>
- https://ha2juo.tistory.com/5<br>
- https://gagadi.tistory.com/35<br>
- https://where-i-am.tistory.com/m/entry/%EB%B9%85%EB%B6%84%EA%B8%B0-2021-%EB%B9%85%EB%8D%B0%EC%9D%B4%ED%84%B0-%EB%B6%84%EC%84%9D%EA%B8%B0%EC%82%AC-2%ED%9A%8C-%EC%8B%A4%EA%B8%B0-%ED%9B%84%EA%B8%B0-%EB%B3%B5%EC%9B%90-%EB%AC%B8%EC%A0%9C<br>
- https://www.kaggle.com/agileteam/tutorial-t1-2-python/notebook

# 작업형 1

## 주어진 데이터셋의 범죄율(crim)에서 상위 10개의 데이터에 대해서 10개 중 최솟값으로 대체한 후, AGE 변수에서 값이 80 이상인 데이터들 중 범죄율의 평균값 구하기
- 보스턴 데이터 https://github.com/selva86/datasets/blob/master/BostonHousing.csv

In [41]:
import pandas as pd

boston = pd.read_csv('boston.csv')
boston_sort = boston.sort_values('crim', ascending = False).reset_index()
del boston_sort['index']
boston_top10 = boston_sort.head(10)
boston_top10

# 상위 10개 crim 열 최솟값으로 대체
crim_min = boston_top10.crim.min()
boston_top10['crim'] = crim_min
boston_sort.head(14)

# age 변수 조건
crim_result = boston_sort[boston_sort['age'] >= 80].crim.mean()
print(crim_result)

5.759386625


<ipython-input-41-6a9d9168c2f3>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boston_top10['crim'] = crim_min


## 주어진 데이터 첫번째 행 부터 순서대로 80%까지의 데이터를 추출 후 'total_bedrooms' 변수의 결측값(NA)을 'total_bedrooms' 변수의 중앙값으로 대체하고 대체 전의 'total_bedrooms' 변수 표준편차값과 대체 후의 'total_bedrooms' 변수 표준편차 값 산출
- 캘리포니아 데이터 https://www.kaggle.com/harrywang/housing?select=housing.csv

In [61]:
california = pd.read_csv('california.csv')

# 80% 까지 추출
row_range = int(california.shape[0] * 0.8)
california_80 = california.iloc[:row_range]

# NA 확인 및 대체
# california_80.isnull().sum()
california_na_rm = california_80.copy()
california_na_rm['total_bedrooms'] = california_80['total_bedrooms'].fillna(california_80['total_bedrooms'].median())

# 대체 전 표준편차 값
print(california_80.total_bedrooms.std())

# 대체 후 표준편차 값
print(california_na_rm.total_bedrooms.std())

435.90057705252616
433.9254297608805


## 주어진 데이터 셋에서 age 컬럼의 이상치를 IQR rule에 의해 탐지하고 이상치들의 합을 구하기(단, 평균으로부터 '표준편차*1.5'를 벗어나는 영역을 이상치라고 판단함)
- 보험료 데이터 https://www.kaggle.com/mirichoi0218/insurance/version/1

In [101]:
# 문제에서 제시된 이상치 탐지법 적용
insurance = pd.read_csv('insurance.csv')
age_mean = insurance.age.mean()
age_std = insurance.age.std()

min_range = age_mean - age_std*1.5
max_range = age_mean + age_std*1.5

outlier_cond = (insurance.age < min_range) | (insurance.age > max_range)
# print(min_range, max_range)
print(insurance.age[outlier_cond].sum())

6928


In [100]:
# 일반적인 IQR 이상치 탐지법 적용
insurance = pd.read_csv('insurance.csv')
Q1 = insurance.age.quantile(.25)
median = insurance.age.median()
Q3 = insurance.age.quantile(.75)

IQR = Q3 - Q1

min_range = Q1 - IQR*1.5
max_range = Q3 + IQR*1.5

outlier_remove_cond = (insurance.age > min_range) & (insurance.age < max_range)
print(min_range, max_range)
insurance.age[outlier_remove_cond]   # 이상치가 아예 존재 x

-9.0 87.0


0       19
1       18
2       28
3       33
4       32
        ..
1333    50
1334    18
1335    18
1336    21
1337    61
Name: age, Length: 1338, dtype: int64

# 작업형 2
고객 구매 데이터를 사용해서 고객이 주문한 물품이 제 시간에 도착여부(Reached.on.Time_Y.N) 예측
- 데이터 https://www.kaggle.com/prachi13/customer-analytics?select=Train.csv

-----------------------------------------------------원본데이터에서 Train, Test 데이터 셋 추출하는 과정이므로 그대로 진행 --

In [109]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name
    
    if null_name != "":
        df[df == null_name] = np.nan
    
    X_train, X_test = train_test_split(df, test_size=0.2, shuffle=True, random_state=2021)
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[id_name, target])
    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[id_name, target])
    return X_train, X_test, y_train, y_test 
    
df = pd.read_csv("./Train.csv")
x_train, x_test, y_train, y_test = exam_data_load(df, 
                                                  target='Reached.on.Time_Y.N', 
                                                  id_name='ID')

x_train.shape, x_test.shape, y_train.shape, y_test.shape

((8799, 10), (2200, 10), (8799, 2), (2200, 2))

In [110]:
df

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N
0,1,D,Flight,4,2,177,3,low,F,44,1233,1
1,2,F,Flight,4,5,216,2,low,M,59,3088,1
2,3,A,Flight,2,2,183,4,low,M,48,3374,1
3,4,B,Flight,3,3,176,4,medium,M,10,1177,1
4,5,C,Flight,2,2,184,3,medium,F,46,2484,1
...,...,...,...,...,...,...,...,...,...,...,...,...
10994,10995,A,Ship,4,1,252,5,medium,F,1,1538,1
10995,10996,B,Ship,4,1,232,5,medium,F,6,1247,0
10996,10997,C,Ship,5,4,242,5,low,F,4,1155,0
10997,10998,F,Ship,5,2,223,6,medium,M,2,1210,0


In [103]:
# 제공된 3개 데이터 (y_test데이터 활용X)
x_train.shape, x_test.shape, y_train.shape

((8799, 10), (2200, 10), (8799, 2))

-----------------------------------------------------여기까지 진행하고 x_train, x_test, y_train 사용 --

In [104]:
x_train.head()

,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms
3999,B,Ship,3,4,134,3,high,F,6,5680
9559,F,Ship,4,3,173,3,medium,M,5,5331
2649,B,Ship,2,1,192,3,high,M,46,3206
4843,F,Ship,6,5,284,4,medium,M,8,5346
9601,F,Flight,3,1,246,3,low,F,10,4707


In [105]:
x_test.head()

,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms
4732,C,Ship,5,5,250,2,medium,M,10,5475
2039,F,Road,5,2,202,3,medium,M,34,3885
5113,F,Ship,5,4,154,2,low,F,5,5528
2360,A,Ship,6,4,209,5,low,F,53,3169
5995,F,Ship,6,1,250,4,low,M,4,1018


In [106]:
y_train.head()

,ID,Reached.on.Time_Y.N
3999,4000,0
9559,9560,1
2649,2650,1
4843,4844,1
9601,9602,1
